In [1]:
#從datatime模組引入與日期相關的類別包含datatime和timeddelta
from datetime import datetime, timedelta

#創造開始日和截止日的string
start_date = "2019-2-01"
stop_date = "2019-3-01"

#strptime 將 datetime string 轉換成object
start = datetime.strptime(start_date, "%Y-%m-%d")
stop = datetime.strptime(stop_date, "%Y-%m-%d")

#用一個while迴圈和timedelta計算日期把開始日到截止日存進一個list裡面
#strftime 將 datetime object 轉換成String
dates = list()
while start <= stop:
    dates.append(start.strftime('%Y%m%d'))
    start = start + timedelta(days=1)

In [2]:
#引入beautifulsoup和requests模組
#requests用於將鏈接轉化成html語言，而BeautifulSoup則用於查找需要的內容。
import requests
from bs4 import BeautifulSoup as bs

In [3]:
#write a function to parse the HTML response, return the data we want(title, content, ...etc)
def process_document(document, date):
    
    #BeautifulSoup的select()作用為選取特定的標籤，類名前加點，id名前加 #，屬性用 [屬性=’xxxx’]，返回類型為list
    #HTML <ul> 標籤用來定義一個無排序的項目清單列表
    #HTML <li> 標籤顯示個別項目的標籤
    nodes = document.select('ul.list > li')
    data = list()

    for li in nodes:

        # check if is empty element
        #select_one()返回值是list的首個
        #<a>定義超連結
        if li.select_one('a') == None:
            continue

        # get link
        #<a> 標籤的 href 屬性用於指定超鏈接目標的網頁地址(URL)。
        li_link = 'http://news.ltn.com.tw/' + li.select_one('a')['href']

        # request for document
        # 使用get()方式下載普通網頁
        # r.text : 輸出網頁 HTML 原始碼   
        li_res = requests.get(li_link)
        li_doc = bs(li_res.text, 'lxml')

        # get date
        li_date = datetime.strptime(date, "%Y%m%d").strftime('%Y-%m-%d')

        #get title
        li_title = li.select_one('p').get_text()

        #get content
        li_content = ""
        for ele in li_doc.select('div.text > p'):
            if not 'appE1121' in ele.get('class', []):
                li_content += ele.get_text()

        # append new row
        #用名為data的list的資料結構儲存dictionary的資料型類
        data.append({
            'date' : li_date,
            'title': li_title,
            'link' : li_link,
            'content' : li_content,
            'tags' : []
        })
        
        #Python 3.x 中，python 直譯器預期的 .py 編碼，預設是 UTF-8，然而在 Windows 中使用記事本編輯文字檔案，預設的編碼是 MS950
        with open('titles.txt', 'a', encoding='utf-8') as file: 
            file.write('\n'+li_title) 
            
    return data

In [4]:
#Crawl over the news on the site, store the data in variable "all_data" .

cnt = 0
all_data = list()
for date in dates:
    print('start crawling :', date)
    res = requests.get('https://news.ltn.com.tw/list/newspaper/politics/' + date)
    doc = bs(res.text, 'lxml')
    data = process_document(doc, date)
    all_data += data

start crawling : 20190201
start crawling : 20190202
start crawling : 20190203
start crawling : 20190204
start crawling : 20190205
start crawling : 20190206
start crawling : 20190207
start crawling : 20190208
start crawling : 20190209
start crawling : 20190210
start crawling : 20190211
start crawling : 20190212
start crawling : 20190213
start crawling : 20190214
start crawling : 20190215
start crawling : 20190216
start crawling : 20190217
start crawling : 20190218
start crawling : 20190219
start crawling : 20190220
start crawling : 20190221
start crawling : 20190222
start crawling : 20190223
start crawling : 20190224
start crawling : 20190225
start crawling : 20190226
start crawling : 20190227
start crawling : 20190228
start crawling : 20190301


In [5]:
#Check the result
all_data[:5]

[{'date': '2019-02-01',
  'title': '嚴德發：想要和平 先做好戰爭準備',
  'link': 'http://news.ltn.com.tw/news/politics/paper/1265470',
  'content': '〔記者涂鉅旻／台北報導〕中國國家主席習近平在今年初發表談話，再度提及一國兩制、不放棄武力犯台主張，國防部長嚴德發昨也對全體國軍表示，英國前首相邱吉爾曾在二戰時引述古羅馬人物名言稱：「想要和平，先要做好戰爭的準備。」在面對未來日益嚴峻挑戰，盼國軍全體官兵秉持敬業樂群精神，戮力戰訓本務。嚴德發在昨日播出的政戰節目「莒光園地」發表談話表示，習近平在今年元旦藉著紀念「告台灣同胞書」四十週年場合發表談話，表明不放棄以武力犯台，蔡英文總統立刻提出「四個必須」及「三道防護網」，宣示帶領國人拚民生、護民主、守主權，迎向挑戰。嚴德發表示，三軍統帥對國軍的指導很明確，就是持續精實戰力，捍衛國家發展與維護百姓福祉，讓國軍成為自由民主永遠的守護者。他也援引前英國首相邱吉爾在二戰期間，引述古羅馬人物韋格蒂烏斯（Vegetius）的名言說：「想要和平，先要做好戰爭的準備。」畢竟軍隊存在的目的，就是要以堅實的戰力維護和平。嚴德發表示，面對未來日益嚴峻的挑戰，他希望全體官兵秉持敬業樂群精神，戮力戰訓本務，莫忘初衷，善盡保國衛民的神聖使命，確保人民能夠享有免於恐懼的安和樂利生活，這樣才能不負國人的期許。',
  'tags': []},
 {'date': '2019-02-01',
  'title': '習近平嗆武統 歐洲議會批中挺台',
  'link': 'http://news.ltn.com.tw/news/politics/paper/1265471',
  'content': '〔記者彭琬馨／台北報導〕中國領導人習近平上月初發表對台談話，不承諾放棄對台動武。歐洲議會三十日罕見公開討論兩岸政治議題，歐盟執委會官員除發表談話籲兩岸維持現狀外，也強調將與台灣在不同領域緊密合作。歐洲議會三十日在比利時布魯塞爾召開全會，會中邀請歐盟執委會針對「中國與台灣兩岸關係的最新發展」發表看法。歐盟執委會人道援助及危機管理執委斯蒂里亞尼迪斯表示，近年兩岸關係趨於緊張，歐盟對此感到遺憾，今年中國領導人習近平發表對台談話後，歐盟迅速回應希望兩岸能

In [6]:
#Save as pkl file
import pickle

#wb:以二進制格式打開一個文件只用於寫入。
with open('liberty_times.pkl', 'wb') as f:
    pickle.dump(all_data, f)

In [7]:
#Turn it into pandas dataframe
import pandas as pd
pd.DataFrame(all_data)[['date', 'title', 'link', 'content', 'tags']].head()

,date,title,link,content,tags
0,2019-02-01,嚴德發：想要和平 先做好戰爭準備,http://news.ltn.com.tw/news/politics/paper/126...,〔記者涂鉅旻／台北報導〕中國國家主席習近平在今年初發表談話，再度提及一國兩制、不放棄武力犯台...,[]
1,2019-02-01,習近平嗆武統 歐洲議會批中挺台,http://news.ltn.com.tw/news/politics/paper/126...,〔記者彭琬馨／台北報導〕中國領導人習近平上月初發表對台談話，不承諾放棄對台動武。歐洲議會三十...,[]
2,2019-02-01,總統遭國台辦謾罵 吳釗燮回嗆瘋子,http://news.ltn.com.tw/news/politics/paper/126...,〔記者彭琬馨／台北報導〕農曆春節前夕，總統蔡英文至全台各地勞軍行程，遭中國國台辦發言人馬曉光...,[]
3,2019-02-01,禁中國資通產品原則 政院延至年後宣布,http://news.ltn.com.tw/news/politics/paper/126...,〔記者李欣芳／台北報導〕行政院要禁止中央機關採購中國資通訊等產品，原規劃一月底發布「各機關使...,[]
4,2019-02-01,賀台灣關係法40週年 佛訥籲美與台簽FTA,http://news.ltn.com.tw/news/politics/paper/126...,〔記者蘇永耀、吳佳穎／台北報導〕蔡英文總統昨日接見美國智庫「傳統基金會」創辦人佛訥（Edwi...,[]


In [8]:
with open('titles.txt', 'r', encoding='utf-8') as Read: 
    print(Read.read()) 


嚴德發：想要和平 先做好戰爭準備
習近平嗆武統 歐洲議會批中挺台
總統遭國台辦謾罵 吳釗燮回嗆瘋子
禁中國資通產品原則 政院延至年後宣布
賀台灣關係法40週年 佛訥籲美與台簽FTA
AIT賀歲片 改編爆紅神曲
「禮讓韓國瑜當選市長」 退休副分局長放假訊息
挪用勞軍款被控貪瀆》買皮蛋送長官 少將無罪定讞
美日駐台使節 同日拜訪韓國瑜
去年至今 勞動基金賺227億
勞動基金1月由負轉正 彌平去年虧損
阿輝伯出院 除夕夜四代圍爐
總統立委選舉 明年合併舉行
吳敦義︰王若選總統會祝福
吳：兩岸各自治理 主權無法分割
選舉史頭一遭 驗票翻盤 候選人竟未判勝選
韓國瑜︰講到2020 像孫悟空聽到緊箍咒
朱立倫人馬 又一人進韓市府
總統期許海巡弟兄 繼續努力阻絕豬瘟
總統警衛室主任 曾維聰突調職
謠言終結站》國道拋錨 中油免費拖吊? 中油:無此服務
「跟著小英去拜年」 總統預告年節行程
黨內人士：韓國瑜比朱立倫好輔選
藍營拱選總統 韓國瑜：好好過年別胡思亂想
基層挺韓升溫 朱立倫：我會更努力爭取支持
立委拜年小物 藍推開運錢母、長腿照月曆
立委拜年小物 綠贈原葉茶包、招財環保袋
一週大事（1月27日至2月2日）
謠言終結站》用大數據追稅？ 財政部：減少徵納成本
台美共推 印太宗教自由對話下月登場
小英許3願 秀廣東話、潮州話拜年
除民怨 蘇揆快速反應
藍營太陽逐大位 年後大戲上場
吳桂華愛畫畫 也為老闆王金平「搭橋」
謠言終結站》沒核電2021年將缺電？ 經部︰顛倒是非
本週熱點》朱立倫預計10日赴美參訪
治安亮紅燈 高雄一夜3砍殺
台大改變國際抗生素指南
兩岸軍力加速失衡// 中國今年軍費5.4兆 台灣16倍
幾乎全癱到能走 傘兵秦良丰轉院
508克寶寶 住保溫箱151天// 媽媽說故事伴兒長大
朱會韓︰兩人絕不會是對手
立院答詢 蘇︰不怕朱韓合 怕朱重啟核四
看「朱韓會」 王稱自己也會安排見柯P
吳今邀連馬朱王茶敘
《焦點透視》大選指標
韓將赴中交流 蘇揆：別被極權政治框架框住
蘇：做好準備 才能維護和平
嚴德發：有能力迫使敵人犯台失敗
軍方：不做軍備競賽 力拚不對稱戰力
美智庫：制衡美國 中國急擴充軍備
共軍轟六部署廣東 距台僅450公里
一國兩制是關愛？ 陸委會斥中國荒謬
治安亮紅燈 高雄一夜3砍殺
台大改變國際抗生素指南
兩岸軍力加速失衡// 中國今年軍費5.4兆 台